# SQL Database in AWS

The databse table structures need modified. Currently everything was tossed into the database without relationships between the tables. as a way to just save the data and make sure it was clean enough for the system. 
The task now is to:
- Extract the data from the database and review it's structure.
- Restructure the tables to show the relationships within the data. 

In [1]:
import os
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, inspect
from sqlalchemy.ext.automap import automap_base
from config import *

db_address = db_type + db_user + db_pass + db_project + db_aws

In [2]:
engine = create_engine(db_address)

In [3]:
conn = engine.connect()

In [4]:
wine_reviews = [x for x in conn.execute("select * from wine_reviews;")]
wine_reviews

[(2, 'Rainstorm', 2013.0, None, 'Pinot Grigio', 87, 'White', 'Rainstorm 2013 Pinot Gris (Willamette Valley)', 'Willamette Valley', 'Willamette Valley', 'Oregon', 'US', '$14.0', 'Tart and snappy, the flavors of lime flesh and rind dominate. Some green pineapple pokes through, with crisp acidity underscoring the flavors. The wine was all stainless-steel fermented.', 'Wine Enthusiest'),
 (4, 'Sweet Cheeks', 2012.0, "Vintner's Reserve Wild Child Block", 'Pinot Noir', 87, 'Red', "Sweet Cheeks 2012 Vintner's Reserve Wild Child Block Pinot Noir (Willamette Valley)", 'Willamette Valley', 'Willamette Valley', 'Oregon', 'US', '$65.0', "Much like the regular bottling from 2012, this comes across as rather rough and tannic, with rustic, earthy, herbal characteristics. Nonetheless, if you think of it as a pleasantly unfussy country wine, it's a good companion to a hearty winter stew.", 'Wine Enthusiest'),
 (21, 'Acrobat', 2013.0, None, 'Pinot Noir', 87, 'Red', 'Acrobat 2013 Pinot Noir (Oregon)', No

In [5]:
## Column Names in order for wine_reviews
wine_review_columns= ("review_id", "winery", "vintage", 'designation', 'variety', 'score', 'category', 'title', 'ava', 'region', 'province', 'country', 'release_price', 'review_text', 'source')
wine_review_columns

('review_id',
 'winery',
 'vintage',
 'designation',
 'variety',
 'score',
 'category',
 'title',
 'ava',
 'region',
 'province',
 'country',
 'release_price',
 'review_text',
 'source')

In [6]:
vineyards = [x for x in conn.execute("select * from vineyards;")]
vineyards

[('ChIJY_YvLqs_lVQRQ1JGKPCC-qA', 'Beaux Frères', 'OPERATIONAL', '(503) 537-1137', 'By Appointment Only, 15155 NE North Valley Rd, Newberg, OR 97132, USA', 'https://maps.google.com/?cid=11599727758225592899', 'http://www.beauxfreres.com/', 4.9, 57, ' 15155 NE North Valley Rd', ' Newberg', 'OR', '97132', ' USA', None),
 ('ChIJb07wO0IUlVQRpT7hr_IPjrc', 'Ruby Vineyard', 'OPERATIONAL', '(503) 628-7829', '30088 SW Egger Rd, Hillsboro, OR 97123, USA', 'https://maps.google.com/?cid=13226526690641002149', 'http://www.rubyvineyard.com/', 4.7, 83, '30088 SW Egger Rd', ' Hillsboro', 'OR', '97123', ' USA', None),
 ('ChIJ0837TqdBlVQR8w_X_SPvVx4', 'Dobbes Family Estate', 'OPERATIONAL', '(503) 538-1141', '240 SE 5th St, Dundee, OR 97115, USA', 'https://maps.google.com/?cid=2186479081973288947', 'http://dobbesfamilyestate.com/', 4.7, 94, '240 SE 5th St', ' Dundee', 'OR', '97115', ' USA', None),
 ('ChIJ22vW2sjFolQRohtq-192_d0', 'FireThorn Vineyard', 'OPERATIONAL', '(206) 391-6709', '75049 Snow Rd, Echo,

In [7]:
## Column Names in order for vineyards, need to drop the last column, it is empty
vineyards_columns = ('place_id', 'business_name', 'business_status', 'phone_number', 'formatted_address', "gmaps_url", 'website', 'rating', 'total_reviews', 'stree_address', 'city', 'province', 'zipcode', 'country', 'empty' )
vineyards_columns

('place_id',
 'business_name',
 'business_status',
 'phone_number',
 'formatted_address',
 'gmaps_url',
 'website',
 'rating',
 'total_reviews',
 'stree_address',
 'city',
 'province',
 'zipcode',
 'country',
 'empty')

In [8]:
vineyard_reviews = [x for x in conn.execute("select * from vineyard_reviews;")]
vineyard_reviews

[(0, 'Kate S.', 'https://www.google.com/maps/contrib/100465257518510170399/reviews', 'en', 4, '5 months ago', "We must have had an off day when we visited. The visit was all inside even though it was a lovely warm day, the inside is dark and not very warm + we ... (979 characters truncated) ... o pay for the tasting ($100 out of pocket)-- then take anything home. Would have preferred to taste some more recent releases that could be cellared.", 1653419759, False, 'ChIJY_YvLqs_lVQRQ1JGKPCC-qA'),
 (1, 'Joel Barrett', 'https://www.google.com/maps/contrib/116620073433236231932/reviews', 'en', 5, 'a year ago', 'Great old vine wines here! You are bound to find several you’ll love. Beautiful property with views of Mt Hood 60+ miles away.', 1632516637, False, 'ChIJb07wO0IUlVQRpT7hr_IPjrc'),
 (2, 'Susan Whitmarsh', 'https://www.google.com/maps/contrib/107392748716269509723/reviews', 'en', 5, '6 months ago', 'We enjoyed visiting, sitting in our own warm little bubble.  Everyone was friendly and w

In [9]:
## Column Names in order for vineyard_reviews
vineyard_reviews_columns= ('reviw_id', "author_name", 'review_link','language', 'time_description', 'review_text', 'utc_time', 'translated', 'place_id' )
vineyard_reviews_columns

('reviw_id',
 'author_name',
 'review_link',
 'language',
 'time_description',
 'review_text',
 'utc_time',
 'translated',
 'place_id')

In [10]:
geometry = [x for x in conn.execute("select * from geometry;")]
geometry

[(45.34762389999999, -123.0936571, 45.3489524302915, -123.0923680197085, 45.3462544697085, -123.0950659802915, 'ChIJY_YvLqs_lVQRQ1JGKPCC-qA'),
 (45.40038819999999, -122.9869325, 45.40193678029149, -122.9850769697085, 45.39923881970849, -122.9877749302915, 'ChIJb07wO0IUlVQRpT7hr_IPjrc'),
 (45.2795302, -123.0078417, 45.28064498029149, -123.0067110197085, 45.27794701970849, -123.0094089802915, 'ChIJ0837TqdBlVQR8w_X_SPvVx4'),
 (45.72194710000001, -119.1918038, 45.72329608029151, -119.1904548197085, 45.72059811970851, -119.1931527802915, 'ChIJ22vW2sjFolQRohtq-192_d0'),
 (43.115009, -123.39355, 43.1167391802915, -123.3922426197085, 43.1140412197085, -123.3949405802915, 'ChIJZS1JIJ05xFQRBC3XPLxiy0o'),
 (44.82778169999999, -123.0123551, 44.8291171802915, -123.0110208197085, 44.8264192197085, -123.0137187802915, 'ChIJEZsIboQIwFQRNy3yOiTKe_Y'),
 (45.4441154, -123.1941181, 45.4454538302915, -123.1929544697085, 45.4427558697085, -123.1956524302915, 'ChIJn5MXyjQ9lVQRvfzTTYvgfSI'),
 (45.3870262, -12

In [11]:
## Column Names in order for geometry
geometry_columns = ('center_lat',"center_lon", 'northeast_lat', 'northeast_lon', 'southwest_lat', 'southwest_lon', 'place_id')
geometry_columns

('center_lat',
 'center_lon',
 'northeast_lat',
 'northeast_lon',
 'southwest_lat',
 'southwest_lon',
 'place_id')

- The review_text column for both the vineyard_reviews and the wine_reviews was truncated by pandas when inserting the data into the SQL Database. I am not happy about this and am glad we have the csv's as backups. This needs brought up to the team at the next meeting. 
- I am also sad the url for the wine_reviews was not saved. I dropped the column in wine_reviews during ETL because only about 5 to 10% were saved.  Will have to go back and grab those at a later date.
- Since there are wineries with multiple vineyards a unique winery or "brand" list must be created and applied to both the vineyards table and unique wine table.

In [12]:
conn.close()

In [13]:
## join geometry and vineyards into one dataframe, common column = 'place_id'
vineyards_df = pd.DataFrame(vineyards, columns=vineyards_columns)
geometry_df = pd.DataFrame(geometry, columns=geometry_columns)


In [14]:
## using the wine_reviews, create a unique wine list and 
## extract the unique wineries for a common column with google_vineyards
wine_reviews_df = pd.DataFrame(wine_reviews, columns=wine_review_columns)
wine_reviews_df

,review_id,winery,vintage,designation,variety,score,category,title,ava,region,province,country,release_price,review_text,source
0,2,Rainstorm,2013.0,None,Pinot Grigio,87,White,Rainstorm 2013 Pinot Gris (Willamette Valley),Willamette Valley,Willamette Valley,Oregon,US,$14.0,"Tart and snappy, the flavors of lime flesh and...",Wine Enthusiest
1,4,Sweet Cheeks,2012.0,Vintner's Reserve Wild Child Block,Pinot Noir,87,Red,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Willamette Valley,Willamette Valley,Oregon,US,$65.0,"Much like the regular bottling from 2012, this...",Wine Enthusiest
2,21,Acrobat,2013.0,None,Pinot Noir,87,Red,Acrobat 2013 Pinot Noir (Oregon),None,Oregon Other,Oregon,US,$20.0,"A sleek mix of tart berry, stem and herb, alon...",Wine Enthusiest
3,35,Erath,2010.0,Hyland,Pinot Noir,86,Red,Erath 2010 Hyland Pinot Noir (McMinnville),McMinnville,Willamette Valley,Oregon,US,$50.0,As with many of the Erath 2010 vineyard design...,Wine Enthusiest
4,41,Hawkins Cellars,2009.0,None,Pinot Noir,86,Red,Hawkins Cellars 2009 Pinot Noir (Willamette Va...,Willamette Valley,Willamette Valley,Oregon,US,$22.0,"A stiff, tannic wine, this slowly opens and br...",Wine Enthusiest
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6606,1233,Knudsen,2015.0,None,Chardonnay,90,None,Knudsen 2015 Chardonnay Dundee Hills,Dundee Hills,Willamette Valley,Oregon,US,$45,"Crisp and harmonious, with lemon blossom and a...",Wine Spectator Magazine Issue Web Only - 2017
6607,1234,Ponzi,2015.0,Reserve,Chardonnay,90,None,Ponzi 2015 Chardonnay Willamette Valley Reserve,Willamette Valley,Willamette Valley,Oregon,US,$40,"Graceful and polished, with expressive pear, s...","Wine Spectator Magazine Issue Jul 31, 2018"
6608,1235,Raptor Ridge,2015.0,Gran Moraine Vineyard,Chardonnay,90,None,Raptor Ridge 2015 Chardonnay Yamhill-Carlton D...,Yamhill-Carlton District,Willamette Valley,Oregon,US,$45,"Light and polished, showing pretty pear, pinea...",Wine Spectator Magazine Issue Web Only - 2017
6609,1236,Roco,2015.0,Marsh Estate Vineyards,Chardonnay,90,None,Roco 2015 Chardonnay Yamhill-Carlton District ...,Yamhill-Carlton District,Willamette Valley,Oregon,US,$60,"Floral and harmonious, with delicate tropical ...",Wine Spectator Magazine Issue Web Only - 2018


In [15]:
import re
vineyards_df.set_index("place_id", inplace=True)

In [16]:
## seperate out the most common brand types
cellar = vineyards_df.loc[[i for i in vineyards_df.index if "Cellar" in vineyards_df['business_name'][i]],'business_name']
cellar_pattern = r'((\S+\s*)+(Cellars?))'
# cellar = cellar.str.extract(cellar_pattern)
# cellar

In [17]:
## seperate out the most common brand types
winery = vineyards_df.loc[[i for i in vineyards_df.index if "Winery" in vineyards_df['business_name'][i]],'business_name']
winery.drop([i for i in winery.index if i in cellar.index], axis=0, inplace=True)
winery_pattern = r'((\S+\s*)+(?:Winery))'
# winery = winery.str.extract(winery_pattern)[0]
# winery

In [18]:
## seperate out the most common brand types
vine = vineyards_df.loc[[i for i in vineyards_df.index if "Vineyard" in vineyards_df['business_name'][i]],'business_name']
vine.drop([i for i in vine.index if i in cellar.index], axis=0, inplace=True)
vine.drop([i for i in vine.index if i in winery.index], axis=0, inplace=True)

vine_pattern = r'((\S+\s*)+(Vineyards?))'
# vine = vine.str.extract(vine_pattern)[0]

# concat into one series
brand = pd.concat([cellar, winery, vine])
# brand


In [19]:
special = vineyards_df.loc[[i for i in vineyards_df.index if i not in brand.index],'business_name']
p_pattern = special.loc[[i for i in special.index if ' (' in special[i]],]
p_pattern = p_pattern.apply(lambda x : x[:x.index('(')-1])
brand = pd.concat([brand, p_pattern])

In [20]:
special = vineyards_df.loc[[i for i in vineyards_df.index if i not in brand.index],'business_name']
dash_pattern = r'((\S+\s*)+(?=\s-\s))'
dash = special.loc[[i for i in special.index if ' - ' in special[i]],]
# dash = dash.str.extract(dash_pattern)[0]
brand = pd.concat([brand, dash])

In [21]:
special = vineyards_df.loc[[i for i in vineyards_df.index if i not in brand.index],'business_name']
estate = special.loc[[i for i in special.index if "Estate" in special[i]],]
estate_pattern = r'((\S+\s*)+(Estates?))'
# estate = estate.str.extract(estate_pattern)
brand = pd.concat([brand, estate])

In [22]:
special = vineyards_df.loc[[i for i in vineyards_df.index if i not in brand.index],'business_name']
special

place_id
ChIJY_YvLqs_lVQRQ1JGKPCC-qA                                 Beaux Frères
ChIJkxfV22gOllQRsg6ngubxivI                               The Pines 1852
ChIJs-4c3txIlVQR6UVxrpn_h7U    Ransom Wine Co. & Distillery Tasting Room
ChIJG9hkQ53IuFQRYkSXxXQKcKk                    The Good Drop Wine Shoppe
ChIJh35nmr3jwFQRuSSHefRzR9s                              Northwest Vines
                                                 ...                    
ChIJqYQgPMQ5lVQRNM6cGXLECi0                    Carlton Hill Wine Company
ChIJrUrHOAJBlVQRpVQFxJCE6gA                                  Ayoub Wines
ChIJhfpqHHY5wFQRbjZipl371Hk                               Lumos Wine Co.
ChIJBeD_m8R6lVQRoiirtxSFVoY                     Christopher Bridge Wines
ChIJdW4z1FCGz1QRpEgO5GsmqE8                                Schultz Wines
Name: business_name, Length: 125, dtype: object

In [23]:
wine_reviews_df = pd.DataFrame(wine_reviews, columns=wine_review_columns)
from_reviews = list(wine_reviews_df['winery'].unique())
from_reviews.sort()
from_reviews

['1789 Wines',
 '18401 Cellars',
 '2Hawk',
 '3Fools',
 '60 Souls',
 'A Blooming Hill Vineyard',
 'A to Z',
 'A to Z Wineworks',
 'Abacela',
 'Abbey Creek',
 'Aberrant Cellars',
 'Abiqua Wind',
 'Acrobat',
 'Adelsheim',
 'Agate Ridge',
 'Airlie',
 'Alchemist',
 'Alchemy Wine Productions',
 'Alexana',
 'Alloro',
 'Amalie Robert',
 'Amity',
 'Amity Vineyards',
 'Anam Cara',
 'Anam Cara Cellars',
 'Ancien',
 'Ancient Cellars',
 'Andrew Rich',
 'Angela Estate',
 'Animale',
 'Anne Amie',
 'Antica Terra',
 'Antiquum Farm',
 'Apolloni',
 'Arbor Brook',
 'Arcane Cellars',
 'Archer',
 'Archery Summit',
 'Ardiri',
 'Argyle',
 'Arterberry Maresh',
 'Artisanal',
 'Ashland Vineyards',
 'Atticus',
 'Aubichon',
 'Aubichon Cellars',
 'Authentique',
 'Ayoub',
 'BKB',
 'Battle Creek',
 'Beacon Hill',
 'Bear Creek',
 'Beaux Frères',
 'Bell',
 'Belle Vallée',
 'Benton-Lane',
 'Benziger',
 'Beran',
 'Beresan',
 'Bergström',
 'Bernard Machado',
 'Besadien',
 'Bethel Heights',
 'Big Table Farm',
 'Bishop Cree

In [24]:
duplicates = set()
for x in from_reviews:
    for n in from_reviews:
        if (x in n) and (x != n):
            if x == 'Ancien':
                continue
            else:
                duplicates.add(x)

for x in duplicates:
    from_reviews.remove(x)

from_reviews
            

['1789 Wines',
 '18401 Cellars',
 '2Hawk',
 '3Fools',
 '60 Souls',
 'A Blooming Hill Vineyard',
 'A to Z Wineworks',
 'Abacela',
 'Abbey Creek',
 'Aberrant Cellars',
 'Abiqua Wind',
 'Acrobat',
 'Adelsheim',
 'Agate Ridge',
 'Airlie',
 'Alchemist',
 'Alchemy Wine Productions',
 'Alexana',
 'Alloro',
 'Amalie Robert',
 'Amity Vineyards',
 'Anam Cara Cellars',
 'Ancien',
 'Ancient Cellars',
 'Andrew Rich',
 'Angela Estate',
 'Animale',
 'Anne Amie',
 'Antica Terra',
 'Antiquum Farm',
 'Apolloni',
 'Arbor Brook',
 'Arcane Cellars',
 'Archery Summit',
 'Ardiri',
 'Argyle',
 'Arterberry Maresh',
 'Artisanal',
 'Ashland Vineyards',
 'Atticus',
 'Aubichon Cellars',
 'Authentique',
 'Ayoub',
 'BKB',
 'Battle Creek',
 'Beacon Hill',
 'Bear Creek',
 'Beaux Frères',
 'Belle Vallée',
 'Benton-Lane',
 'Benziger',
 'Beran',
 'Beresan',
 'Bergström',
 'Bernard Machado',
 'Besadien',
 'Bethel Heights',
 'Big Table Farm',
 'Bishop Creek Cellars',
 'Björnson',
 'Black Magnolia',
 'BlackCap',
 'Blakeslee

In [26]:
save  = []
for i in special.index:
    for place in from_reviews:
        if place in special[i]:
            if place == 'pino':
                continue
                
            save.append((i, place))
            
save    

[('ChIJY_YvLqs_lVQRQ1JGKPCC-qA', 'Beaux Frères'),
 ('ChIJs-4c3txIlVQR6UVxrpn_h7U', 'Ransom'),
 ('ChIJA2FQfmxBlVQRmuceABxatAw', 'Day'),
 ('ChIJUa7JFuU8lVQREp7TUm_IIME', 'Big Table Farm'),
 ('ChIJ7cZV901qlVQR512cTmU2Nik', 'Chehalem'),
 ('ChIJgflueyBBlVQRjvQePp_Yts4', 'Arterberry Maresh'),
 ('ChIJvcqlCJM_lVQRlVAiOhWcvfc', 'Nicolas-Jay'),
 ('ChIJC8FEpTHppFQRxUVfOt0xZvk', 'Copper Belt'),
 ('ChIJk8AyJCMJllQR5nx396gJhgc', 'Phelps Creek'),
 ('ChIJ-3NYYMJIlVQRVHh3ddXKYZ8', 'Dominio IV'),
 ('ChIJ5fhwQ2I4lVQRphdyVnYkmUA', 'Flâneur'),
 ('ChIJB3rkytNBlVQRJeGnnZKdTpc', 'Longplay'),
 ('ChIJV1ZPvUtBlVQRu6PnF0iWLnc', 'Archery Summit'),
 ('ChIJD8TDcjk-lVQRFKIJquVRyUA', 'Eminent Domaine'),
 ('ChIJdwZiaalrlVQRNU5wOfFmNsU', 'Et Fille'),
 ('ChIJHSsTIGM4lVQRZS9GSCYz2Ic', 'Siltstone'),
 ('ChIJp3eA550OllQR5gujRSmfdmA', 'Viento'),
 ('ChIJWRS9kbVGlVQRNktsfS8kMyw', 'Durant'),
 ('ChIJN8t-U-qqwVQRzqB7-njzzxw', 'Brigadoon'),
 ('ChIJw86okx9twFQRSp83utHpxZM', 'Delfino'),
 ('ChIJUwmbTltrolQR-8kMGCvVUBk', 'Devona'),
 ('

In [45]:
from_reviews = pd.DataFrame(save, columns=['place_id', 'name']).set_index('place_id')
from_reviews = from_reviews['name']
from_reviews

place_id
ChIJY_YvLqs_lVQRQ1JGKPCC-qA              Beaux Frères
ChIJs-4c3txIlVQR6UVxrpn_h7U                    Ransom
ChIJA2FQfmxBlVQRmuceABxatAw                       Day
ChIJUa7JFuU8lVQREp7TUm_IIME            Big Table Farm
ChIJ7cZV901qlVQR512cTmU2Nik                  Chehalem
ChIJgflueyBBlVQRjvQePp_Yts4         Arterberry Maresh
ChIJvcqlCJM_lVQRlVAiOhWcvfc               Nicolas-Jay
ChIJC8FEpTHppFQRxUVfOt0xZvk               Copper Belt
ChIJk8AyJCMJllQR5nx396gJhgc              Phelps Creek
ChIJ-3NYYMJIlVQRVHh3ddXKYZ8                Dominio IV
ChIJ5fhwQ2I4lVQRphdyVnYkmUA                   Flâneur
ChIJB3rkytNBlVQRJeGnnZKdTpc                  Longplay
ChIJV1ZPvUtBlVQRu6PnF0iWLnc            Archery Summit
ChIJD8TDcjk-lVQRFKIJquVRyUA           Eminent Domaine
ChIJdwZiaalrlVQRNU5wOfFmNsU                  Et Fille
ChIJHSsTIGM4lVQRZS9GSCYz2Ic                 Siltstone
ChIJp3eA550OllQR5gujRSmfdmA                    Viento
ChIJWRS9kbVGlVQRNktsfS8kMyw                    Durant
ChIJN8t-U-qqwVQRzqB

In [48]:
pd.concat([brand, from_reviews])


place_id
ChIJGwkzz5lPlVQRrC17L0dfWfE                             Bravuro Cellars Vineyard
ChIJAQAcT5FBolQR0X-6RLStHik                                       Tulpen Cellars
ChIJrWcAWigBwFQRpNXuw3xLJok                                    Chemeketa Cellars
ChIJFxlq4W45xFQRofB1FANAgqA                                           HV Cellars
ChIJQXewxnhDolQRj4yrsvTkKmg    Patterson Cellars Walla Walla Tasting Room & V...
                                                     ...                        
ChIJX2kc1TNBlVQR2ByNtkV6kkk                               Domaine Drouhin Oregon
ChIJqYQgPMQ5lVQRNM6cGXLECi0                                         Carlton Hill
ChIJrUrHOAJBlVQRpVQFxJCE6gA                                                Ayoub
ChIJhfpqHHY5wFQRbjZipl371Hk                                                Lumos
ChIJdW4z1FCGz1QRpEgO5GsmqE8                                              Schultz
Name: name, Length: 552, dtype: object

In [36]:
brand.dropna(axis=0, inplace=True)